## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Severo-Kurilsk,RU,50.6789,156.1250,30.09,68,99,10.47,overcast clouds
1,1,Bang Saphan,TH,11.2126,99.5117,76.06,91,80,3.04,broken clouds
2,2,Plyussa,RU,58.4308,29.3654,34.30,99,100,2.15,overcast clouds
3,3,San Cristobal,VE,7.7669,-72.2250,71.58,60,33,0.60,scattered clouds
4,4,Karup,DK,56.3067,9.1684,41.58,81,13,8.05,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
raw_preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
raw_preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bang Saphan,TH,11.2126,99.5117,76.06,91,80,3.04,broken clouds
3,3,San Cristobal,VE,7.7669,-72.2250,71.58,60,33,0.60,scattered clouds
5,5,Rikitea,PF,-23.1203,-134.9692,79.99,71,100,13.73,overcast clouds
9,9,Kavieng,PG,-2.5744,150.7967,85.33,63,61,7.61,broken clouds
11,11,Touros,BR,-5.1989,-35.4608,79.38,83,64,9.17,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
raw_preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = raw_preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                267
City                   267
Country                267
Lat                    267
Lng                    267
Max Temp               267
Humidity               267
Cloudiness             267
Wind Speed             267
Current Description    267
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bang Saphan,TH,76.06,broken clouds,11.2126,99.5117,
3,San Cristobal,VE,71.58,scattered clouds,7.7669,-72.2250,
5,Rikitea,PF,79.99,overcast clouds,-23.1203,-134.9692,
9,Kavieng,PG,85.33,broken clouds,-2.5744,150.7967,
11,Touros,BR,79.38,broken clouds,-5.1989,-35.4608,
12,Puerto Escondido,MX,83.23,broken clouds,15.8500,-97.0667,
13,Hobart,AU,70.05,few clouds,-42.8794,147.3294,
15,Tual,ID,82.06,moderate rain,-5.6667,132.7500,
18,Sao Joao Da Barra,BR,76.64,scattered clouds,-21.6403,-41.0511,
19,Atuona,PF,79.21,broken clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Index Error Counter 
i_error = 0
no_hotels = []

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        i_error += 1
        no_hotels.append(hotel_df.loc[index, "City"])

print(f"Among {len(hotel_df)} cities, {i_error} cities had no hotels within 5,000 meters. These cities are:")
print(no_hotels)

Among 267 cities, 18 cities had no hotels within 5,000 meters. These cities are:
['Gamba', 'Taoudenni', 'Ouadda', 'Mitu', 'Luebo', 'Koungheul', 'Beloha', 'Tessalit', 'Latung', 'Fonte Boa', 'Yaan', 'Mbamba', 'Gao', 'Solenzo', 'Obo', 'Tadine', 'Tambacounda', 'Wawa']


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# df = df[df.column_name != value: (Source: https://www.statology.org/pandas-drop-rows-with-value/)
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.count()

City                   249
Country                249
Max Temp               249
Current Description    249
Lat                    249
Lng                    249
Hotel Name             249
dtype: int64

In [10]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bang Saphan,TH,76.06,broken clouds,11.2126,99.5117,Ladawan Boutique Hotel
3,San Cristobal,VE,71.58,scattered clouds,7.7669,-72.2250,Pirineos
5,Rikitea,PF,79.99,overcast clouds,-23.1203,-134.9692,People ThankYou
9,Kavieng,PG,85.33,broken clouds,-2.5744,150.7967,Nusa Island Retreat
11,Touros,BR,79.38,broken clouds,-5.1989,-35.4608,INN NEW HORIZON


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))